In [12]:
import os
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_experimental.tools import PythonAstREPLTool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from Modules import logging
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from Modules.messages import display_message_tree
from Modules.messages import stream_response

In [2]:
# DATA_PATH
DATA_PATH = "data/"

# Logging
logging.langsmith("Search for documents App")

LangSmith 추적을 시작합니다.
[프로젝트명]
Search for documents App


In [3]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [4]:
# 저장된 데이터를 로드
db = FAISS.load_local(
    folder_path="faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

In [5]:
retriever = db.as_retriever()

In [6]:
prompt = PromptTemplate.from_template(
"""
    <CACH_PROMPT>
    "To answer the FAQ-style question '{question}', search for the most relevant document.
    Summarize only the essential content to ensure the user can quickly understand the answer."

    1. Key Guidelines
    Optimal Document Search: Search for the document most relevant to the user’s question based on key terms, and evaluate the content’s direct relevance to the query.
    Maintain Accuracy and Reliability: Ensure the retrieved document is based on accurate and verified information to secure the user’s trust.
    Exclude Unnecessary Information: Omit content unrelated to the question or overly detailed information, providing only the essential points the user needs.
    
    2. Visualization Tasks
    Provide Concise Summaries: Summarize the answer concisely in a text block that aligns with an FAQ format.
    Highlight Key Terms: When necessary, bold essential keywords or use emphasis to enable the user to grasp the core points quickly.
    Paragraph Structure: If the response is lengthy, divide the information into paragraphs to fit the FAQ style appropriately.
    
    3. Response Format
    Concise and Clear Answers: Structure the answer in a brief, clear manner, using 2-3 sentences so the user can understand it quickly.
    Direct and Summary Format: If a direct answer is possible, use a short format to provide clarity.
    Add Examples: If needed, include brief examples or code snippets to aid understanding, but keep examples concise.
   
    4. Response Language
    User Language: Return answers in the same language as the query to maintain language consistency.
    Minimize Technical Jargon: Simplify or briefly explain technical terms to enhance user comprehension.
    Polite and Friendly Tone: Use a friendly tone to ensure users feel comfortable with the response.
    
    5. Task Steps
    Keyword Matching: Extract key terms from the user’s question and match with the most relevant sections in the FAQ documents.
    Document Filtering and Sorting: Rank the selected documents based on relevance to the user’s question, filtering out those with low relevance.
    Document Summarization and Response Formatting: Summarize the key content of the selected document in an FAQ format and apply any necessary formatting to enhance user-friendliness.
    Final Review: Ensure the response is clear, essential, and directly addresses the question.
    Return the Answer: Provide the summarized response to the user, including any additional information or links as needed.
    
    #QUESTION:
    {question}

    #Context:
    {context}
    
    #Answer in Korean:
"""
    
)

In [7]:
llm = ChatOpenAI(model_name= "gpt-4o", temperature=0)

In [8]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    |llm
    |StrOutputParser()
)

In [13]:
question = "python은 어떤 언어야?"
response = chain.stream(question)
stream_response(response)

Python은 **고급 프로그래밍 언어**로, 주로 **인터프리터 언어**로 사용됩니다. 이는 Python 코드가 실행될 때 즉시 해석되어 실행된다는 것을 의미합니다. Python은 **간결하고 읽기 쉬운 문법**을 가지고 있어 초보자부터 전문가까지 널리 사용됩니다. 또한, 다양한 분야에서 활용 가능하며, 특히 **웹 개발, 데이터 분석, 인공지능** 등에서 인기가 높습니다.

In [ ]:
# messages = [{"role": "user","content":"python은 어떤 언어야?"}]

# response = chain.stream(messages=messages)


In [ ]:
# print(response.messages[-1]["content"])

AttributeError: 'generator' object has no attribute 'messages'

In [ ]:
display_message_tree(response=messages)


root: <generator object RunnableSequence.stream at 0x00000242284FDC60>
